In [46]:
import pandas as pd
import numpy as np
import re

In [47]:
SI1 = pd.read_csv('SI_Reactions.txt',sep = '\t')
SI1[-20:]

,ID,Protein,Gene,Reaction,Regulatory logic,References,Notes
112,PYRex,Pyruvate exchange,NaN,PYRxt ->,NaN,NaN,NaN
113,RIBex,Ribose exchange,NaN,RIBxt ->,NaN,NaN,NaN
114,SUCCex,Succinate exchange,NaN,SUCCxt ->,NaN,NaN,NaN
115,E. Regulatory Proteins,NaN,NaN,NaN,NaN,NaN,NaN
116,NaN,Aerobic/Anaerobic response regulator,arcA,NaN,active IF not (O2xt),"23,30",NaN
117,NaN,Catabolite activator protein,cra (fruR),NaN,active IF not (surplus FDP or F6P)*,19,Surplus FDP or F6P = if not (VFBP > 0 and not ...
118,NaN,Catabolite repressor protein,crp,NaN,action is complex and highlighted in italics a...,"22,50",NaN
119,NaN,Dicarboxylate response regulator,dcuR,NaN,active IF DcuS,18,NaN
120,NaN,Dicarboxylate response sensor,dcuS,NaN,active IF (SUCCxt),18,NaN
121,NaN,Fatty acid/Acetate response regulator,fadR,NaN,active IF (GLCxt) or not (ACxt),2,NaN


In [48]:
SI1.loc[SI1.ID == 'FUMCR']

,ID,Protein,Gene,Reaction,Regulatory logic,References,Notes
22,FUMCR,Fumarase C,fumC,FUM <-> MAL,IF (superoxide radicals),26,Accumulation of superoxide radicals in the cel...


In [49]:
#superoxide radicals are not modelled
SI1.loc[SI1.ID == 'FUMCR','Regulatory logic'] = np.nan
SI1.loc[SI1.ID == 'FUMCR']

,ID,Protein,Gene,Reaction,Regulatory logic,References,Notes
22,FUMCR,Fumarase C,fumC,FUM <-> MAL,NaN,26,Accumulation of superoxide radicals in the cel...


In [50]:
#crp regulation is complex and modelled separately
SI1.drop(SI1[SI1.Gene == 'crp'].index, inplace=True)
SI2 = pd.read_csv('SI_Metabolites.txt',sep = '\t')
SI2 = SI2.sort_values(by = 'id')
SI2

,id,name
12,AC,Acetate
16,ACCOA,Acetyl-CoA
20,ACTP,Acetyl-phosphate
24,ACxt,External acetate
28,ADP,Adenosine diphosphate
...,...,...
59,UDPGAL,UDP Galactose
63,UTP,Uridine triphosphate
67,X5P,Xylulose-5-phosphate
44,bDGLAC,b-D-Galactose


In [51]:
SI1.iloc[118]

ID                                               NaN
Protein             Dicarboxylate response regulator
Gene                                            dcuR
Reaction                                         NaN
Regulatory logic                      active IF DcuS
References                                        18
Notes                                            NaN
Name: 119, dtype: object

# Reformulate lumped gene IDs, change some genes and obtain a gene list

In [52]:
# Reformulation of "lumped" IDs
SI1.loc[SI1.ID == 'ACEE', 'Gene'] = 'aceE, aceF, lpdA'
SI1.loc[SI1.ID == 'ATPAR', 'Gene'] = 'atpA, atpB, atpC, atpD, atpE, atpF, atpG, atpH, atpI'
SI1.loc[SI1.ID == 'CYDA', 'Gene'] = 'cydA, cydB'
SI1.loc[SI1.ID == 'CYOA', 'Gene'] = 'cyoA, cyoB, cyoC, cyoD'
SI1.loc[SI1.ID == 'FDNG', 'Gene'] = 'fdnG, fdnH, fdnI'
SI1.loc[SI1.ID == 'FDOH', 'Gene'] = 'fdoI, fdoH, fdoG'
SI1.loc[SI1.ID == 'FRDA', 'Gene'] = 'frdA, frdB, frdC, frdD'
SI1.loc[SI1.ID == 'GLPA', 'Gene'] = 'glpA, glpB, glpC'
SI1.loc[SI1.ID == 'NUOA', 'Gene'] = 'nuoA, nuoB, nuoE, nuoF, nuoG, nuoH, nuoI, nuoJ, nuoK, nuoL, nuoM, nuoN'
SI1.loc[SI1.ID == 'PFLA', 'Gene'] = 'pflA, pflB'
SI1.loc[SI1.ID == 'PFLC', 'Gene'] = 'pflC, pflD'
SI1.loc[SI1.ID == 'PNTA1', 'Gene'] = 'pntA, pntB'
SI1.loc[SI1.ID == 'PNTA2', 'Gene'] = 'pntA, pntB'
SI1.loc[SI1.ID == 'SDHA1', 'Gene'] = 'sdhA, sdhB, sdhC, sdhD'
SI1.loc[SI1.ID == 'SDHA2', 'Gene'] = 'sdhA, sdhB, sdhC, sdhD'
SI1.loc[SI1.ID == 'SUCA', 'Gene'] = 'sucA, sucB, lpdA'
SI1.loc[SI1.ID == 'SUCCR', 'Gene'] = 'sucC, sucD'
SI1.loc[SI1.ID == 'GLCPTS', 'Gene'] = 'ptsG, ptsH, ptsI, crr'
SI1.loc[SI1.ID == 'PIUP2R', 'Gene'] = 'pitA, pitB'
SI1.loc[SI1.ID == 'RIBUPR', 'Gene'] = 'rbsA, rbsB, rbsC, rbsD'

In [53]:
# manual curation
SI1.loc[SI1.Protein == 'Catabolite activator protein', 'Gene'] = 'cra' # 'cra (fruR)' - both names refer to the same gene
SI1.loc[SI1.ID == 'GLCUP', 'Gene'] = 'galP' # 'galP, etc.' - galP is sufficient
SI1.loc[SI1.ID == 'PCKA', 'Gene'] = 'pck' # 'pckA'
SI1.loc[SI1.ID == 'SFCA', 'Gene'] = 'maeA' # 'sfcA'
SI1.loc[SI1.ID == 'GPMBR', 'Gene'] = 'gpmA' # 'gpmB' does not exist, gpmA catalyses the same reaction

In [54]:
genes_list = []

for index, value in SI1['Gene'].items():
    if isinstance(value, str):
        for g in value.split(', '):
            if g not in genes_list:
                genes_list.append(g)

In [55]:
genes_list

['aceA',
 'aceB',
 'aceE',
 'aceF',
 'lpdA',
 'ackA',
 'acnA',
 'acnB',
 'acs',
 'adhE',
 'adk',
 'atpA',
 'atpB',
 'atpC',
 'atpD',
 'atpE',
 'atpF',
 'atpG',
 'atpH',
 'atpI',
 'cydA',
 'cydB',
 'cyoA',
 'cyoB',
 'cyoC',
 'cyoD',
 'dld',
 'eno',
 'fba',
 'fbp',
 'fdnG',
 'fdnH',
 'fdnI',
 'fdoI',
 'fdoH',
 'fdoG',
 'frdA',
 'frdB',
 'frdC',
 'frdD',
 'fumA',
 'fumB',
 'fumC',
 'galE',
 'galK',
 'galM',
 'galT',
 'galU',
 'gapA',
 'glk',
 'glpA',
 'glpB',
 'glpC',
 'glpD',
 'glpK',
 'gltA',
 'gnd',
 'gpmA',
 'gpsA',
 'icdA',
 'lacZ',
 'maeB',
 'mdh',
 'ndh',
 'nuoA',
 'nuoB',
 'nuoE',
 'nuoF',
 'nuoG',
 'nuoH',
 'nuoI',
 'nuoJ',
 'nuoK',
 'nuoL',
 'nuoM',
 'nuoN',
 'pck',
 'pfkA',
 'pfkB',
 'pflA',
 'pflB',
 'pflC',
 'pflD',
 'pgi',
 'pgk',
 'pgl',
 'pgm',
 'pntA',
 'pntB',
 'ppa',
 'ppc',
 'ppsA',
 'pta',
 'pykA',
 'pykF',
 'rbsK',
 'rpe',
 'rpiA',
 'rpiB',
 'sdhA',
 'sdhB',
 'sdhC',
 'sdhD',
 'maeA',
 'sucA',
 'sucB',
 'sucC',
 'sucD',
 'talA',
 'talB',
 'tktA',
 'tktB',
 'tpiA',
 '

In [56]:
SI2

,id,name
12,AC,Acetate
16,ACCOA,Acetyl-CoA
20,ACTP,Acetyl-phosphate
24,ACxt,External acetate
28,ADP,Adenosine diphosphate
...,...,...
59,UDPGAL,UDP Galactose
63,UTP,Uridine triphosphate
67,X5P,Xylulose-5-phosphate
44,bDGLAC,b-D-Galactose


# Splitting metabolites into intracellular and extracellular ones

In [57]:
# Split SI2 into intra- and extracellular metabolites

# add column with boolean variable
SI2['external'] = 1
SI2['external'] = SI2['name'].str.contains("External")

# split dataset
extracellular = SI2.query("external")
intracellular = SI2.query("external == False")

In [58]:
intracellular

,id,name,external
12,AC,Acetate,False
16,ACCOA,Acetyl-CoA,False
20,ACTP,Acetyl-phosphate,False
28,ADP,Adenosine diphosphate,False
32,AKG,a-Ketoglutarate,False
...,...,...,...
59,UDPGAL,UDP Galactose,False
63,UTP,Uridine triphosphate,False
67,X5P,Xylulose-5-phosphate,False
44,bDGLAC,b-D-Galactose,False


In [59]:
# drop boolean column
intracellular = intracellular.drop(['external'], axis=1)

# add attributes
intracellular["compartment"] = 'cytosol'
intracellular["constant"] = 'false'
intracellular["boundaryCondition"] = 'false'
intracellular["hasOnlySubstanceUnits"] = 'true'
intracellular["ram:speciesType"] = 'metabolite'

In [60]:
# drop boolean column
extracellular = extracellular.drop(['external'], axis=1)

In [61]:
# abundant species
extracellular.at[extracellular['id'] == 'CO2xt', 'initialAmount'] = 10000
extracellular.at[extracellular['id'] == 'HEXT', 'initialAmount'] = 10000
extracellular.at[extracellular['id'] == 'PIxt', 'initialAmount'] = 10000

# potential product are initially zero
extracellular.at[extracellular['id'] == 'ETHxt', 'initialAmount'] = 0
extracellular.at[extracellular['id'] == 'FORxt', 'initialAmount'] = 0
extracellular.at[extracellular['id'] == 'LACxt', 'initialAmount'] = 0
extracellular.at[extracellular['id'] == 'LCTSxt', 'initialAmount'] = 0
extracellular.at[extracellular['id'] == 'PYRxt', 'initialAmount'] = 0

# carbon sources and O2: initial amount depends on the scenario; aerobic glucose as default:
extracellular.at[extracellular['id'] == 'ACxt', 'initialAmount'] = 0
extracellular.at[extracellular['id'] == 'GLCxt', 'initialAmount'] = 100
extracellular.at[extracellular['id'] == 'GLxt', 'initialAmount'] = 0
extracellular.at[extracellular['id'] == 'O2xt', 'initialAmount'] = 10000
extracellular.at[extracellular['id'] == 'RIBxt', 'initialAmount'] = 0
extracellular.at[extracellular['id'] == 'SUCCxt', 'initialAmount'] = 0

In [62]:
# add attributes
extracellular["compartment"] = 'extracellular'
extracellular["constant"] = 'false'
extracellular["boundaryCondition"] = 'false' # not false for every species
extracellular["hasOnlySubstanceUnits"] = 'true'
extracellular["ram:speciesType"] = 'extracellular'

In [63]:
extracellular

,id,name,initialAmount,compartment,constant,boundaryCondition,hasOnlySubstanceUnits,ram:speciesType
24,ACxt,External acetate,0.0,extracellular,false,false,true,extracellular
64,CO2xt,External carbon dioxide,10000.0,extracellular,false,false,true,extracellular
9,ETHxt,External ethanol,0.0,extracellular,false,false,true,extracellular
33,FORxt,External Formate,0.0,extracellular,false,false,true,extracellular
69,GLCxt,External glucose,100.0,extracellular,false,false,true,extracellular
75,GLxt,External glycerol,0.0,extracellular,false,false,true,extracellular
2,HEXT,External H+,10000.0,extracellular,false,false,true,extracellular
14,LACxt,External lactate,0.0,extracellular,false,false,true,extracellular
22,LCTSxt,External Lactose,0.0,extracellular,false,false,true,extracellular
50,O2xt,External Oxygen,10000.0,extracellular,false,false,true,extracellular


# Regulatory Proteins

In [64]:
# create copy of first part of SI
rp = SI1[['ID', 'Protein', 'Gene']].copy()
# keep rows without ID
rp = rp[pd.isnull(rp['ID'])]
rp = rp.drop(['ID'], axis=1)
rp

,Protein,Gene
116,Aerobic/Anaerobic response regulator,arcA
117,Catabolite activator protein,cra
119,Dicarboxylate response regulator,dcuR
120,Dicarboxylate response sensor,dcuS
121,Fatty acid/Acetate response regulator,fadR
122,Aerobic/Anaerobic response regulator,fnr
123,Galactose operon repressor,galR
124,Galactose operon repressor,galS
125,Glycerol response regulator,glpR
126,Fatty acid/Acetate response regulator,iclR


In [65]:
# define function for create protein names (uppercase) from gene names (lowercase)
def repl_func(m):
    """process regular expression match groups for word upper-casing problem"""
    return m.group(1) + m.group(2).upper()

rp['id_protein'] = rp['Gene'].map(lambda element: re.sub("(^|\s)(\S)", repl_func, element))
# rp id = 'RP_' + protein id
rp["id"] = 'RP_' + rp["id_protein"]
rp = rp[['id', 'Protein', 'Gene']] # order columns
rp.loc[rp.id == 'RP_Fnr', 'id'] = 'RP_FNR'
rp = rp.rename(columns={"Protein": "name"})

C:\Users\koush\anaconda3\envs\FBA\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\Users\koush\anaconda3\envs\FBA\lib\site-packages\pandas\core\indexing.py:692: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


In [66]:
# add attributes
rp["compartment"] = 'cytosol'
rp["constant"] = 'false'
rp["boundaryCondition"] = 'false'
rp["hasOnlySubstanceUnits"] = 'true'
rp["initialAmount"] = 'NaN'
rp["ram:biomassPercentage"] = 0.0
rp["ram:molecularWeight"] = None # are set later
rp["ram:objectiveWeight"] = None # are set later
rp["ram:speciesType"] = 'enzyme'
rp["degradationRate"] = 0.2

# Quota

In [67]:
quota = SI1[['Reaction']].loc[SI1['Protein'] == "Biomass production flux"]
quota

,Reaction
99,41.25 ATP + 3.54 NAD + 18.22 NADPH + 0.2 G6P +...


In [68]:
biomass_rxn = quota['Reaction'].values
biomass_rxn[0].split()

['41.25',
 'ATP',
 '+',
 '3.54',
 'NAD',
 '+',
 '18.22',
 'NADPH',
 '+',
 '0.2',
 'G6P',
 '+',
 '0.07',
 'F6P',
 '+',
 '0.89',
 'R5P',
 '+',
 '0.36',
 'E4P',
 '+',
 '0.12',
 'T3P1',
 '+',
 '1.49',
 'PG3',
 '+',
 '0.51',
 'PEP',
 '+',
 '2.83',
 'PYR',
 '+',
 '3.74',
 'ACCOA',
 '+',
 '1.78',
 'OA',
 '+',
 '1.07',
 'AKG',
 '->',
 '3.74',
 'COA',
 '+',
 '41.25',
 'ADP',
 '+',
 '41.25',
 'PI',
 '+',
 '3.54',
 'NADH',
 '+',
 '18.22',
 'NADP',
 '+',
 '1',
 'Biomass']

In [69]:
stoich = []
spec = []
product = False

for s in biomass_rxn[0].split():
    if '-' in s:
        product = True
    try:
        s = float(s)
        if product:
            stoich.append(s)
        else:
            stoich.append(-s)
    except ValueError:
        if '+' not in s and '-' not in s:
            spec.append(s)

In [70]:
stoich

[-41.25,
 -3.54,
 -18.22,
 -0.2,
 -0.07,
 -0.89,
 -0.36,
 -0.12,
 -1.49,
 -0.51,
 -2.83,
 -3.74,
 -1.78,
 -1.07,
 3.74,
 41.25,
 41.25,
 3.54,
 18.22,
 1.0]

In [71]:
spec

['ATP',
 'NAD',
 'NADPH',
 'G6P',
 'F6P',
 'R5P',
 'E4P',
 'T3P1',
 'PG3',
 'PEP',
 'PYR',
 'ACCOA',
 'OA',
 'AKG',
 'COA',
 'ADP',
 'PI',
 'NADH',
 'NADP',
 'Biomass']

In [72]:
mw_mol = [503.15, 662.42, 741.38, 258.12, 258.12, 228.09, 198.07, 170.06, 183.03, 165.02, 87.05, 805.54, 130.06, 144.08, 763.50, 424.18, 94.98, 663.42, 740.38, 1000]
mw_mmol = [mw/1000 for mw in mw_mol]
biomass = pd.DataFrame({'id': spec, 'stoichiometry': stoich, 'molecular weight [g/mmol]': mw_mmol})
biomass['weight [g]'] = biomass['stoichiometry'] * biomass['molecular weight [g/mmol]']
# calculate weight of biomass
educt_surplus = biomass['weight [g]'].sum() - 1.0 # subtract biomass 'weight'
biomass_weight = -educt_surplus # new biomass weight equals educt surplus
# set weight of 'biomass', which will be reformulated as Quota_rest
biomass.at[biomass['id'] == 'Biomass', 'biomass_weight'] = biomass_weight
biomass

,id,stoichiometry,molecular weight [g/mmol],weight [g],biomass_weight
0,ATP,-41.25,0.50315,-20.754938,NaN
1,NAD,-3.54,0.66242,-2.344967,NaN
2,NADPH,-18.22,0.74138,-13.507944,NaN
3,G6P,-0.20,0.25812,-0.051624,NaN
4,F6P,-0.07,0.25812,-0.018068,NaN
5,R5P,-0.89,0.22809,-0.203000,NaN
6,E4P,-0.36,0.19807,-0.071305,NaN
7,T3P1,-0.12,0.17006,-0.020407,NaN
8,PG3,-1.49,0.18303,-0.272715,NaN
9,PEP,-0.51,0.16502,-0.084160,NaN


In [73]:
#Fraction of proteins is set to 0.55 (for details, see report), so the 'remaining' quota will make up 45 %
#No proteins modelled,setting biomass percent to 100
# add attributes for quota 'rest'
quota["ID"] = 'Quota_rest'
quota["ram:molecularWeight"] = biomass_weight
quota["ram:objectiveWeight"] = biomass_weight
quota["ram:biomassPercentage"] = 1
quota["degradationRate"] = 0.001
quota["initialAmount"] = 1
quota

,Reaction,ID,ram:molecularWeight,ram:objectiveWeight,ram:biomassPercentage,degradationRate,initialAmount
99,41.25 ATP + 3.54 NAD + 18.22 NADPH + 0.2 G6P +...,Quota_rest,0.864801,0.864801,1,0.001,1


In [74]:
quota['compartment'] = 'cytosol'
quota['ram:molecularWeight'] = 1.0
quota['ram:objectiveWeight'] = 1.0
quota['Gene'] = 'NaN'
quota

,Reaction,ID,ram:molecularWeight,ram:objectiveWeight,ram:biomassPercentage,degradationRate,initialAmount,compartment,Gene
99,41.25 ATP + 3.54 NAD + 18.22 NADPH + 0.2 G6P +...,Quota_rest,1.0,1.0,1,0.001,1,cytosol,NaN


# Metabolic reactions and Uptake Reactions

In [75]:
# create copy of first part of SI
reactions = SI1[['ID', 'Protein', 'Gene', 'Reaction']].copy()
# delete rows without Reaction
reactions.dropna(subset=['Reaction'], inplace=True)
# delete exchange reactions
#reactions = reactions[~reactions.Protein.str.contains('exchange')]
# delete maintenance and biomass reactions
reactions.drop(reactions[reactions.ID == 'ATPM'].index, inplace=True)
reactions.drop(reactions[reactions.ID == 'VGRO'].index, inplace=True)

In [76]:
reactions[-20:]

,ID,Protein,Gene,Reaction
91,PYRUPR,Pyruvate transport,NaN,PYRxt + HEXT <-> PYR
92,RIBUPR,Ribose transport,"rbsA, rbsB, rbsC, rbsD",RIBxt + ATP -> RIB + ADP + PI
93,DCTAR,Succinate transport,dctA,SUCCxt + HEXT <-> SUCC
94,DCUAR,Succinate transport,dcuA,SUCCxt + HEXT <-> SUCC
95,DCUBR,Succinate transport,dcuB,SUCCxt + HEXT <-> SUCC
96,DCUC,Succinate efflux,dcuC,SUCC -> SUCCxt +HEXT
101,ACex,Acetate exchange,NaN,ACxt ->
102,CO2ex,Carbon dioxide exchange,NaN,CO2xt ->
103,ETHex,Ethanol exchange,NaN,ETHxt ->
104,FORex,Formate exchange,NaN,FORxt ->


# Maintenance Reaction

In [77]:
# filter maintenance reaction from SI
main_df = SI1.filter(SI1[SI1.ID == 'ATPM'].index, axis = 0)

# Biomass Reaction

In [78]:
# filter biomass reaction from SI
bio_df = SI1.filter(SI1[SI1.ID == 'VGRO'].index, axis = 0)

# Events and algebraic rules

In [79]:
events = pd.DataFrame()
rules = pd.DataFrame()

# Carbon Catabolite Repression and Crp

In [80]:
# Glc_neg defined by Event low_/high_Glc below
# Lactose
events = events.append({'id':'low_LCTS_aux', 'variable':['GLCxt', 'LCTSxt'], 'relation':'lt', 'threshold':'theta_xt', 'assignment':['Lcts_neg_aux'], 'value':['1']}, ignore_index=True)
events = events.append({'id':'high_LCTS_aux', 'variable':['GLCxt', 'LCTSxt'], 'relation':'geq', 'threshold':'theta_xt', 'assignment':['Lcts_neg_aux'], 'value':['0']}, ignore_index=True)
# Ribose
events = events.append({'id':'low_RIB_aux', 'variable':['GLCxt', 'LCTSxt', 'RIBxt'], 'relation':'lt', 'threshold':'theta_xt', 'assignment':['Rib_neg_aux'], 'value':['1']}, ignore_index=True)
events = events.append({'id':'high_RIB_aux', 'variable':['GLCxt', 'LCTSxt', 'RIBxt'], 'relation':'geq', 'threshold':'theta_xt', 'assignment':['Rib_neg_aux'], 'value':['0']}, ignore_index=True)
# Glycerol
events = events.append({'id':'low_GL_aux', 'variable':['GLCxt', 'LCTSxt', 'RIBxt', 'GLxt'], 'relation':'lt', 'threshold':'theta_xt', 'assignment':['Gl_neg_aux'], 'value':['1']}, ignore_index=True)
events = events.append({'id':'high_GL_aux', 'variable':['GLCxt', 'LCTSxt', 'RIBxt', 'GLxt'], 'relation':'geq', 'threshold':'theta_xt', 'assignment':['Gl_neg_aux'], 'value':['0']}, ignore_index=True)
# Lactate / Pyruvate
events = events.append({'id':'low_LacPyr_aux', 'variable':['GLCxt', 'LCTSxt', 'RIBxt', 'GLxt', 'LACxt', 'PYRxt'], 'relation':'lt', 'threshold':'theta_xt', 'assignment':['LacPyr_neg_aux'], 'value':['1']}, ignore_index=True)
events = events.append({'id':'high_LacPyr_aux', 'variable':['GLCxt', 'LCTSxt', 'RIBxt', 'GLxt', 'LACxt', 'PYRxt'], 'relation':'geq', 'threshold':'theta_xt', 'assignment':['LacPyr_neg_aux'], 'value':['0']}, ignore_index=True)
# Succinate / Ethanol
events = events.append({'id':'low_SuccEth_aux', 'variable':['GLCxt', 'LCTSxt', 'RIBxt', 'GLxt', 'LACxt', 'PYRxt', 'SUCCxt', 'ETHxt'], 'relation':'lt', 'threshold':'theta_xt', 'assignment':['SuccEth_neg_aux'], 'value':['1']}, ignore_index=True)
events = events.append({'id':'high_SuccEth_aux', 'variable':['GLCxt', 'LCTSxt', 'RIBxt', 'GLxt', 'LACxt', 'PYRxt', 'SUCCxt', 'ETHxt'], 'relation':'geq', 'threshold':'theta_xt', 'assignment':['SuccEth_neg_aux'], 'value':['0']}, ignore_index=True)
# All Carbon
events = events.append({'id':'low_carbon_neg', 'variable':['GLCxt', 'LCTSxt', 'RIBxt', 'GLxt', 'LACxt', 'PYRxt', 'SUCCxt', 'ETHxt', 'ACxt', 'FORxt'], 'relation':'lt', 'threshold':'theta_xt', 'assignment':['carbon_neg', 'carbon'], 'value':['1', '0']}, ignore_index=True)
events = events.append({'id':'high_carbon_neg', 'variable':['GLCxt', 'LCTSxt', 'RIBxt', 'GLxt', 'LACxt', 'PYRxt', 'SUCCxt', 'ETHxt', 'ACxt', 'FORxt'], 'relation':'geq', 'threshold':'theta_xt', 'assignment':['carbon_neg', 'carbon'], 'value':['0', '1']}, ignore_index=True)


In [81]:
# GLCPTS
events = events.append({'id':'low_Mlc', 'variable':['RP_Mlc'], 'relation':'lt', 'threshold':'theta_RP', 'assignment':['Mlc_neg'], 'value':['1']}, ignore_index=True)
events = events.append({'id':'high_Mlc', 'variable':['RP_Mlc'], 'relation':'geq', 'threshold':'theta_RP', 'assignment':['Mlc_neg'], 'value':['0']}, ignore_index=True)
# Cra_neg defined below
rules = rules.append({'variable': 'Glcpts_aux1', 'indicators': ['carbon', 'Mlc_neg'], 'relation': '<and/>'}, ignore_index=True)
rules = rules.append({'variable': 'Glcpts_aux2', 'indicators': ['carbon', 'Cra_neg'], 'relation': '<and/>'}, ignore_index=True)
rules = rules.append({'variable': 'qual_con_GLCPTS_bar', 'indicators': ['Glcpts_aux1', 'Glcpts_aux2'], 'relation': '<or/>'}, ignore_index=True)

# Galactose
events = events.append({'id':'low_GalRS', 'variable':['RP_GalR', 'RP_GalS'], 'relation':'lt', 'threshold':'theta_RP', 'assignment':['GalRS_neg'], 'value':['1']}, ignore_index=True)
events = events.append({'id':'high_GalRS', 'variable':['RP_GalR', 'RP_GalS'], 'relation':'geq', 'threshold':'theta_RP', 'assignment':['GalRS_neg'], 'value':['0']}, ignore_index=True)
rules = rules.append({'variable': 'Gal_aux', 'indicators': ['Glc_neg', 'GalRS_neg'], 'relation': '<and/>'}, ignore_index=True)

# Lactose
events = events.append({'id':'low_LacI', 'variable':['RP_LacI'], 'relation':'lt', 'threshold':'theta_RP', 'assignment':['LacI_neg'], 'value':['1']}, ignore_index=True)
events = events.append({'id':'high_LacI', 'variable':['RP_LacI'], 'relation':'geq', 'threshold':'theta_RP', 'assignment':['LacI_neg'], 'value':['0']}, ignore_index=True)
rules = rules.append({'variable': 'Lcts_aux', 'indicators': ['Glc_neg', 'LacI_neg'], 'relation': '<and/>'}, ignore_index=True)

# Ribose
events = events.append({'id':'low_RbsR', 'variable':['RP_RbsR'], 'relation':'lt', 'threshold':'theta_RP', 'assignment':['RbsR_neg'], 'value':['1']}, ignore_index=True)
events = events.append({'id':'high_RbsR', 'variable':['RP_RbsR'], 'relation':'geq', 'threshold':'theta_RP', 'assignment':['RbsR_neg'], 'value':['0']}, ignore_index=True)
rules = rules.append({'variable': 'Rib_aux', 'indicators': ['Lcts_neg', 'RbsR_neg'], 'relation': '<and/>'}, ignore_index=True)

# Glycerol
# GLPK, GLUPR
events = events.append({'id':'low_GlpR', 'variable':['RP_GlpR'], 'relation':'lt', 'threshold':'theta_RP', 'assignment':['GlpR_neg'], 'value':['1']}, ignore_index=True)
events = events.append({'id':'high_GlpR', 'variable':['RP_GlpR'], 'relation':'geq', 'threshold':'theta_RP', 'assignment':['GlpR_neg'], 'value':['0']}, ignore_index=True)
rules = rules.append({'variable': 'GG_aux', 'indicators': ['Rib_neg', 'GlpR_neg'], 'relation': '<and/>'}, ignore_index=True)

# GLPA (FNR_neg defined below)
rules = rules.append({'variable': 'qual_con_GLPA_bar', 'indicators': ['GG_aux', 'FNR_aux'], 'relation': '<and/>'}, ignore_index=True)
# GLPD
events = events.append({'id':'low_ArcA_GlpR', 'variable':['RP_ArcA', 'RP_GlpR'], 'relation':'lt', 'threshold':'theta_RP', 'assignment':['Arca_GlpR_neg'], 'value':['1']}, ignore_index=True)
events = events.append({'id':'high_ArcA_GlpR', 'variable':['RP_ArcA', 'RP_GlpR'], 'relation':'geq', 'threshold':'theta_RP', 'assignment':['Arca_GlpR_neg'], 'value':['0']}, ignore_index=True)
rules = rules.append({'variable': 'qual_con_GLPD_bar', 'indicators': ['Rib_neg', 'Arca_GlpR_neg'], 'relation': '<and/>'}, ignore_index=True)

# Succinate
# DCTAR (ArcA_neg defined below)
events = events.append({'id':'low_DcuR', 'variable':['RP_DcuR'], 'relation':'lt', 'threshold':'theta_RP', 'assignment':['DcuR_aux'], 'value':['0']}, ignore_index=True)
events = events.append({'id':'high_DcuR', 'variable':['RP_DcuR'], 'relation':'geq', 'threshold':'theta_RP', 'assignment':['DcuR_aux'], 'value':['1']}, ignore_index=True)
rules = rules.append({'variable': 'qual_con_DCTAR_bar', 'indicators': ['LacPyr_neg', 'Arca_neg', 'DcuR_aux'], 'relation': '<and/>'}, ignore_index=True)
# DCUBR
rules = rules.append({'variable': 'qual_con_DCUBR_bar', 'indicators': ['LacPyr_neg', 'FNR_aux', 'DcuR_aux'], 'relation': '<and/>'}, ignore_index=True)

# Acetate (IclR_neg defined below)
rules = rules.append({'variable': 'qual_con_ACS_bar', 'indicators': ['SuccEth_neg', 'IclR_neg'], 'relation': '<and/>'}, ignore_index=True)


# Cra

In [82]:
events = events.append({'id':'low_flux_FBP', 'variable':['FBP'], 'relation':'lt', 'threshold':'theta_v', 'assignment':['FDP_aux'], 'value':['0']}, ignore_index=True)
events = events.append({'id':'high_flux_FBP', 'variable':['FBP'], 'relation':'geq', 'threshold':'theta_v', 'assignment':['FDP_aux'], 'value':['1']}, ignore_index=True)

events = events.append({'id':'low_flux_F6P', 'variable':['TKTA2R', 'TKTB2R', 'TALAR', 'TALBR', 'PGIR'], 'relation':'lt', 'threshold':'theta_v', 'assignment':['F6P_neg'], 'value':['1']}, ignore_index=True)
events = events.append({'id':'high_flux_F6P', 'variable':['TKTA2R', 'TKTB2R', 'TALAR', 'TALBR', 'PGIR'], 'relation':'geq', 'threshold':'theta_v', 'assignment':['F6P_neg'], 'value':['0']}, ignore_index=True)

rules = rules.append({'variable': 'qual_con_cra_bar', 'indicators': ['FDP_aux', 'F6P_neg'], 'relation': '<and/>'}, ignore_index=True)

# PdhR 

In [83]:
events = events.append({'id':'low_flux_MAEB_SFCA', 'variable':['MAEB', 'SFCA'], 'relation':'lt', 'threshold':'theta_v', 'assignment':['PYR_aux1'], 'value':['0']}, ignore_index=True)
events = events.append({'id':'high_flux_MAEB_SFCA', 'variable':['MAEB', 'SFCA'], 'relation':'geq', 'threshold':'theta_v', 'assignment':['PYR_aux1'], 'value':['1']}, ignore_index=True)

events = events.append({'id':'low_flux_DLD1R', 'variable':['DLD1R'], 'relation':'lt', 'threshold':'theta_v', 'assignment':['DLD1R_aux'], 'value':['0']}, ignore_index=True)
events = events.append({'id':'high_flux_DLD1R', 'variable':['DLD1R'], 'relation':'geq', 'threshold':'theta_v', 'assignment':['DLD1R_aux'], 'value':['1']}, ignore_index=True)

events = events.append({'id':'low_flux_PYR_aux', 'variable':['GLCPTS', 'PYKF', 'PYKA', 'DLD2', 'DCTAR', 'DCUAR', 'DCUBR'], 'relation':'lt', 'threshold':'theta_v', 'assignment':['PYR_neg_aux3'], 'value':['1']}, ignore_index=True)
events = events.append({'id':'high_flux_PYR_aux', 'variable':['GLCPTS', 'PYKF', 'PYKA', 'DLD2', 'DCTAR', 'DCUAR', 'DCUBR'], 'relation':'geq', 'threshold':'theta_v', 'assignment':['PYR_neg_aux3'], 'value':['0']}, ignore_index=True)

rules = rules.append({'variable': 'qual_con_pdhR_bar', 'indicators': ['PYR_aux1', 'DLD1R_aux', 'PYR_neg_aux3'], 'relation': '<and/>'}, ignore_index=True)

# ADHER

In [84]:
# O2_neg and Cra_neg are defined below
rules = rules.append({'variable': 'O2_Cra_neg', 'indicators': ['O2_neg', 'Cra_neg'], 'relation': '<and/>'}, ignore_index=True)
rules = rules.append({'variable': 'qual_con_ADHER_bar', 'indicators': ['O2_neg', 'O2_Cra_neg'], 'relation': '<or/>'}, ignore_index=True)

# Proteins controlled by two continuous variables

In [85]:
# ACEB
rules = rules.append({'variable': 'qual_con_ACEB_bar', 'indicators': ['ArcA_neg', 'IclR_neg'], 'relation': '<and/>'}, ignore_index=True)

# CYDA
rules = rules.append({'variable': 'qual_con_CYDA_bar', 'indicators': ['ArcA_aux', 'FNR_neg'], 'relation': '<or/>'}, ignore_index=True)

# FRDA
rules = rules.append({'variable': 'qual_con_FRDA_bar', 'indicators': ['FNR_aux', 'DcuR_aux'], 'relation': '<or/>'}, ignore_index=True)

# DCUC, FORUPR, PFLA, PFLC
rules = rules.append({'variable': 'DFPP_aux', 'indicators': ['ArcA_aux', 'FNR_aux'], 'relation': '<or/>'}, ignore_index=True)


# CYOA, FUMAR, SDHA1, SDHA2
rules = rules.append({'variable': 'CFSS_aux', 'indicators': ['ArcA_neg', 'FNR_neg'], 'relation': '<and/>'}, ignore_index=True)


# FadR
events = events.append({'id':'low_AC', 'variable':['ACxt'], 'relation':'lt', 'threshold':'theta_xt', 'assignment':['AC_neg'], 'value':['1']}, ignore_index=True)
events = events.append({'id':'high_AC', 'variable':['ACxt'], 'relation':'geq', 'threshold':'theta_xt', 'assignment':['AC_neg'], 'value':['0']}, ignore_index=True)
rules = rules.append({'variable': 'qual_con_fadR_bar', 'indicators': ['GLC_aux', 'AC_neg'], 'relation': '<or/>'}, ignore_index=True)

# Proteins controlled by only one continuous variable

In [86]:
# effect of O2 only
events = events.append({'id':'low_O2', 'variable':['O2xt'], 'relation':'lt', 'threshold':'theta_xt', 'assignment':['qual_con_arcA_bar', 'qual_con_fnr_bar', 'O2_neg'], 'value':['1', '1', '1']}, ignore_index=True)
events = events.append({'id':'high_O2', 'variable':['O2xt'], 'relation':'geq', 'threshold':'theta_xt', 'assignment':['qual_con_arcA_bar', 'qual_con_fnr_bar', 'O2_neg'], 'value':['0', '0', '0']}, ignore_index=True)
# effect of SUCC only
events = events.append({'id':'low_SUCC', 'variable':['SUCCxt'], 'relation':'lt', 'threshold':'theta_xt', 'assignment':['qual_con_dcuS_bar'], 'value':['0']}, ignore_index=True)
events = events.append({'id':'high_SUCC', 'variable':['SUCCxt'], 'relation':'geq', 'threshold':'theta_xt', 'assignment':['qual_con_dcuS_bar'], 'value':['1']}, ignore_index=True)
# effect of LCTS only
events = events.append({'id':'low_LCTS', 'variable':['LCTSxt'], 'relation':'lt', 'threshold':'theta_xt', 'assignment':['qual_con_galR_bar', 'qual_con_galS_bar', 'qual_con_lacI_bar'], 'value':['1', '1', '1']}, ignore_index=True)
events = events.append({'id':'high_LCTS', 'variable':['LCTSxt'], 'relation':'geq', 'threshold':'theta_xt', 'assignment':['qual_con_galR_bar', 'qual_con_galS_bar', 'qual_con_lacI_bar'], 'value':['0', '0', '0']}, ignore_index=True)
# effect of GL only
events = events.append({'id':'low_GL', 'variable':['GLxt'], 'relation':'lt', 'threshold':'theta_xt', 'assignment':['qual_con_glpR_bar'], 'value':['1']}, ignore_index=True)
events = events.append({'id':'high_GL', 'variable':['GLxt'], 'relation':'geq', 'threshold':'theta_xt', 'assignment':['qual_con_glpR_bar'], 'value':['0']}, ignore_index=True)
# effect of GLC only
events = events.append({'id':'low_GLC', 'variable':['GLCxt'], 'relation':'lt', 'threshold':'theta_xt', 'assignment':['qual_con_mlc_bar', 'Glc_neg', 'Glc_aux'], 'value':['1', '1', '0']}, ignore_index=True)
events = events.append({'id':'high_GLC', 'variable':['GLCxt'], 'relation':'geq', 'threshold':'theta_xt', 'assignment':['qual_con_mlc_bar', 'Glc_neg', 'Glc_aux'], 'value':['0', '0', '1']}, ignore_index=True)
# effect of RIB only
events = events.append({'id':'low_RIB', 'variable':['RIBxt'], 'relation':'lt', 'threshold':'theta_xt', 'assignment':['qual_con_rbsR_bar', 'qual_con_rpiR_bar'], 'value':['1', '1']}, ignore_index=True)
events = events.append({'id':'high_RIB', 'variable':['RIBxt'], 'relation':'geq', 'threshold':'theta_xt', 'assignment':['qual_con_rbsR_bar', 'qual_con_rpiR_bar'], 'value':['0', '0']}, ignore_index=True)
# effect of RP DcuS only
events = events.append({'id':'low_DcuS', 'variable':['RP_DcuS'], 'relation':'lt', 'threshold':'theta_RP', 'assignment':['qual_con_dcuR_bar'], 'value':['0']}, ignore_index=True)
events = events.append({'id':'high_DcuS', 'variable':['RP_DcuS'], 'relation':'geq', 'threshold':'theta_RP', 'assignment':['qual_con_dcuR_bar'], 'value':['1']}, ignore_index=True)
# effect of RP FadR only
events = events.append({'id':'low_FadR', 'variable':['RP_FadR'], 'relation':'lt', 'threshold':'theta_RP', 'assignment':['qual_con_iclR_bar'], 'value':['0']}, ignore_index=True)
events = events.append({'id':'high_FadR', 'variable':['RP_FadR'], 'relation':'geq', 'threshold':'theta_RP', 'assignment':['qual_con_iclR_bar'], 'value':['1']}, ignore_index=True)

# effect of RP IclR only
events = events.append({'id':'low_IclR', 'variable':['RP_IclR'], 'relation':'lt', 'threshold':'theta_RP', 'assignment':['qual_con_ACEA_bar', 'IclR_neg'], 'value':['1', '1']}, ignore_index=True)
events = events.append({'id':'high_IclR', 'variable':['RP_IclR'], 'relation':'geq', 'threshold':'theta_RP', 'assignment':['qual_con_ACEA_bar', 'IclR_neg'], 'value':['0', '0']}, ignore_index=True)
# effect of RP PdhR only
events = events.append({'id':'low_PdhR', 'variable':['RP_PdhR'], 'relation':'lt', 'threshold':'theta_RP', 'assignment':['qual_con_ACEE_bar', 'qual_con_SUCA_bar'], 'value':['1', '1']}, ignore_index=True)
events = events.append({'id':'high_PdhR', 'variable':['RP_PdhR'], 'relation':'geq', 'threshold':'theta_RP', 'assignment':['qual_con_ACEE_bar', 'qual_con_SUCA_bar'], 'value':['0', '0']}, ignore_index=True)
# effect of RP FNR only
events = events.append({'id':'low_FNR', 'variable':['RP_FNR'], 'relation':'lt', 'threshold':'theta_RP', 'assignment':['qual_con_FDNG_bar', 'qual_con_FUMBR_bar', 'qual_con_NDH_bar', 'FNR_aux', 'FNR_neg'], 'value':['0', '0', '1', '0', '1']}, ignore_index=True)
events = events.append({'id':'high_FNR', 'variable':['RP_FNR'], 'relation':'geq', 'threshold':'theta_RP', 'assignment':['qual_con_FDNG_bar', 'qual_con_FUMBR_bar', 'qual_con_NDH_bar', 'FNR_aux', 'FNR_neg'], 'value':['1', '1', '0', '1', '0']}, ignore_index=True)
# effect of RP ArcA only
events = events.append({'id':'low_arcA', 'variable':['RP_ArcA'], 'relation':'lt', 'threshold':'theta_RP', 'assignment':['qual_con_MDHR_bar', 'ArcA_aux', 'ArcA_neg'], 'value':['1', '0', '1']}, ignore_index=True)
events = events.append({'id':'high_arcA', 'variable':['RP_ArcA'], 'relation':'geq', 'threshold':'theta_RP', 'assignment':['qual_con_MDHR_bar', 'ArcA_aux', 'ArcA_neg'], 'value':['0', '1', '0']}, ignore_index=True)
# effects of Cra only
events = events.append({'id':'low_Cra', 'variable':['RP_Cra'], 'relation':'lt', 'threshold':'theta_RP', 'assignment':['qual_con_PPSA_bar', 'qual_con_PYKF_bar', 'Cra_neg'], 'value':['0', '1', '1']}, ignore_index=True)
events = events.append({'id':'high_Cra', 'variable':['RP_Cra'], 'relation':'geq', 'threshold':'theta_RP', 'assignment':['qual_con_PPSA_bar', 'qual_con_PYKF_bar', 'Cra_neg'], 'value':['1', '0', '0']}, ignore_index=True)
# effect of RpiR only
events = events.append({'id':'low_RpiR', 'variable':['RP_RpiR'], 'relation':'lt', 'threshold':'theta_RP', 'assignment':['qual_con_RPIBR_bar'], 'value':['1']}, ignore_index=True)
events = events.append({'id':'high_RpiR', 'variable':['RP_RpiR'], 'relation':'geq', 'threshold':'theta_RP', 'assignment':['qual_con_RPIBR_bar'], 'value':['0']}, ignore_index=True)
                                                                                                                          

# Qualitative species

In [87]:
# initialize with gene names of regulatory proteins
qualspec = rp[['Gene']].copy()
qualspec['id'] = qualspec['Gene'] + '_bar'
# LACUP: spontaneous, but regulated
qualspec = qualspec.append({'id': 'LACUP_bar'}, ignore_index=True)
qualspec['max'] = 1
qualspec

,Gene,id,max
0,arcA,arcA_bar,1
1,cra,cra_bar,1
2,dcuR,dcuR_bar,1
3,dcuS,dcuS_bar,1
4,fadR,fadR_bar,1
5,fnr,fnr_bar,1
6,galR,galR_bar,1
7,galS,galS_bar,1
8,glpR,glpR_bar,1
9,iclR,iclR_bar,1


# Parameters

In [88]:
parameters = {'epsilon_trans':0.0001, 'theta_xt':0.00001, 'theta_RP': 0.000001, 'theta_v': 0.00001}
fbc_parameters = []
qual_parameters = []

# Write Model as XML

In [89]:
rdfba = True
if rdfba:
    name = 'ecoli_rdFBA'
else:
    name = 'ecoli_dFBA'
f = open(name + '.xml', 'w').close()
f = open(name + '.xml', 'w')    

# First lines

In [90]:
# XML definition
f.write("<?xml version=\"1.0\" encoding=\"UTF-8\"?>\n")
# SBML definition
if rdfba:
    f.write("<sbml xmlns=\"http://www.sbml.org/sbml/level3/version2/core\" level=\"3\" version=\"2\" xmlns:qual=\"http://www.sbml.org/sbml/level3/version1/qual/version1\" qual:required=\"true\" xmlns:fbc=\"http://www.sbml.org/sbml/level3/version1/fbc/version2\" fbc:required=\"false\">\n")
else:
    f.write("<sbml xmlns=\"http://www.sbml.org/sbml/level3/version2/core\" level=\"3\" version=\"2\" xmlns:fbc=\"http://www.sbml.org/sbml/level3/version1/fbc/version2\" fbc:required=\"false\">\n")
# Model Definition
f.write("<model id=\"" + name + "\" name=\"" + name + "\" fbc:strict=\"false\">\n\n")

64

# List of Compartments and Species

In [91]:
f.write("<listOfCompartments>\n  <compartment id=\"extracellular\" constant=\"true\"/>\n  <compartment id=\"cytosol\" constant=\"true\"/>\n</listOfCompartments>\n\n")
f.write("<listOfSpecies>\n")

16

# Metabolites

In [92]:
for index, value in extracellular['id'].items():
    f.write("<species id=\""+value+"\" name=\""+extracellular['name'][index]+"\" compartment=\"extracellular\" initialAmount=\""+str(extracellular['initialAmount'][index])+"\" constant=\"false\" boundaryCondition=\""+extracellular['boundaryCondition'][index]+"\" hasOnlySubstanceUnits=\"true\">\n  <annotation>\n    <ram:RAM xmlns:ram=\"https://www.fairdomhub.org/sops/304\">\n      <ram:species ram:molecularWeight=\"0.0\" ram:objectiveWeight=\"0.0\" ram:biomassPercentage=\"0.0\" ram:speciesType=\"extracellular\"/>\n    </ram:RAM>\n  </annotation>\n</species>\n")
for index, value in intracellular['id'].items():
    f.write("<species id=\""+value+"\" name=\""+intracellular['name'][index]+"\" compartment=\"cytosol\" initialAmount=\"0.0\" constant=\"false\" boundaryCondition=\"false\" hasOnlySubstanceUnits=\"true\">\n  <annotation>\n    <ram:RAM xmlns:ram=\"https://www.fairdomhub.org/sops/304\">\n      <ram:species ram:molecularWeight=\"0.0\" ram:objectiveWeight=\"0.0\" ram:biomassPercentage=\"0.0\" ram:speciesType=\"metabolite\"/>\n    </ram:RAM>\n  </annotation>\n</species>\n")

# Regulatory Proteins

In [93]:
if rdfba:
    for index, value in rp['id'].items():
        f.write("<species id=\""+value+"\" name=\""+rp['name'][index]+"\" compartment=\"cytosol\" initialAmount=\""+str(rp['initialAmount'][index])+"\" constant=\"false\" boundaryCondition=\"false\" hasOnlySubstanceUnits=\"true\">\n  <annotation>\n    <ram:RAM xmlns:ram=\"https://www.fairdomhub.org/sops/304\">\n      <ram:species ram:molecularWeight=\""+str(rp['ram:molecularWeight'][index])+"\" ram:objectiveWeight=\""+str(rp['ram:objectiveWeight'][index])+"\" ram:biomassPercentage=\"0.0\" ram:speciesType=\"enzyme\"/>\n    </ram:RAM>\n  </annotation>\n</species>\n")

# Quota

In [94]:
for index, value in quota['ID'].items():
    f.write("<species id=\""+value+"\" compartment=\"cytosol\" initialAmount=\""+str(quota['initialAmount'][index])+"\" constant=\"false\" boundaryCondition=\"false\" hasOnlySubstanceUnits=\"true\">\n  <annotation>\n    <ram:RAM xmlns:ram=\"https://www.fairdomhub.org/sops/304\">\n      <ram:species ram:molecularWeight=\""+str(quota['ram:molecularWeight'][index])+"\" ram:objectiveWeight=\""+str(quota['ram:objectiveWeight'][index])+"\" ram:biomassPercentage=\""+str(quota['ram:biomassPercentage'][index])+"\" ram:speciesType=\"quota\"/>\n    </ram:RAM>\n  </annotation>\n</species>\n")

In [95]:
f.write("</listOfSpecies>\n\n")

18

# List of Reactions

In [96]:
f.write("<listOfReactions>\n")

18

In [97]:
reactions["fbc:geneProductAssociation"] = np.where(pd.isna(reactions["Gene"]), np.nan, 'E_' + reactions["ID"])
reactions["fbc:maintenanceScaling"] = 0.0
reactions

,ID,Protein,Gene,Reaction,fbc:geneProductAssociation,fbc:maintenanceScaling
0,ACEA,Isocitrate lyase,aceA,ICIT -> GLX + SUCC,E_ACEA,0.0
1,ACEB,Malate synthase A,aceB,ACCOA + GLX -> COA + MAL,E_ACEB,0.0
2,ACEE,Pyruvate dehydrogenase,"aceE, aceF, lpdA",PYR + COA + NAD -> NADH + CO2 + ACCOA,E_ACEE,0.0
3,ACKAR,Acetate kinase A,ackA,ACTP + ADP <-> ATP + AC,E_ACKAR,0.0
4,ACNAR,Aconitase A,acnA,CIT <-> ICIT,E_ACNAR,0.0
...,...,...,...,...,...,...
110,O2ex,Oxygen exchange,NaN,O2xt ->,NaN,0.0
111,PIex,Phosphate exchange,NaN,PIxt ->,NaN,0.0
112,PYRex,Pyruvate exchange,NaN,PYRxt ->,NaN,0.0
113,RIBex,Ribose exchange,NaN,RIBxt ->,NaN,0.0


In [98]:
# stoich = 1.0
stoich_bool = False
for index, value in reactions['ID'].items():
    param_lower = 'T_'+value+'_FBClower'
    param_upper = 'T_'+value+'_FBCupper'
    if rdfba and value == 'LACUP':
        f.write("<reaction id=\""+value+"\" reversible=\"false\" fbc:lowerFluxBound=\"" + param_lower + "\" fbc:upperFluxBound=\"" + param_upper + "\">\n  <annotation>\n    <ram:RAM xmlns:ram=\"https://www.fairdomhub.org/sops/304\">\n      \" ram:maintenanceScaling=\"0.0\"/>\n    </ram:RAM>\n  </annotation>\n")
    elif '<->' in str.split(reactions['Reaction'][0]): # reversible
        f.write("<reaction id=\""+value+"\" reversible=\"true\" fbc:lowerFluxBound=\"" + param_lower + "\" fbc:upperFluxBound=\"" + param_upper + "\">\n  <annotation>\n    <ram:RAM xmlns:ram=\"https://www.fairdomhub.org/sops/304\">\n       \" ram:maintenanceScaling=\"0.0\"/>\n    </ram:RAM>\n  </annotation>\n")
    else:
        f.write("<reaction id=\""+value+"\" reversible=\"false\" fbc:lowerFluxBound=\"" + param_lower + "\" fbc:upperFluxBound=\"" + param_upper + "\">\n  <annotation>\n    <ram:RAM xmlns:ram=\"https://www.fairdomhub.org/sops/304\">\n      \" ram:maintenanceScaling=\"0.0\"/>\n    </ram:RAM>\n  </annotation>\n")
    fbc_parameters.append(param_lower)
    fbc_parameters.append(param_upper)
    if pd.isna(reactions['fbc:geneProductAssociation'][index]): # spontaneous reactions
        pass
    else:
        f.write("  <fbc:geneProductAssociation fbc:id=\""+reactions['fbc:geneProductAssociation'][index]+"\">\n    <fbc:geneProductRef fbc:geneProduct=\""+reactions['fbc:geneProductAssociation'][index]+"\"/>\n  </fbc:geneProductAssociation>\n")
    f.write("  <listOfReactants>\n")
    for g in re.split(r'[+\s]\s*', reactions['Reaction'][index]):
        if '-' in g:
            f.write("  </listOfReactants>\n  <listOfProducts>\n")
        else:
            try:
                stoich = float(g)
                stoich_bool = True
            except ValueError:
                if g == "":
                    pass
                else:
                    if stoich_bool == False:
                        stoich = 1.0
                    f.write("    <speciesReference species=\""+g+"\" stoichiometry=\""+str(stoich)+"\" constant=\"true\"/>\n")
                    stoich_bool = False
    f.write("  </listOfProducts>\n</reaction>\n")

# Maintenance reactions

In [99]:
f.write("<reaction id=\"Maintenance\" name=\"Maintenance\" reversible=\"false\">\n\t<annotation>\n\t\t<ram:RAM xmlns:ram=\"https://www.fairdomhub.org/sops/304\">\n\t\t\t<ram:reaction ram:kcatForward=\"\" ram:kcatBackward=\"\" ram:maintenanceScaling=\"0.14\"/>\n\t\t</ram:RAM>\n\t</annotation>\n\t<listOfReactants>\n\t\t<speciesReference species=\"ATP\" stoichiometry=\"1\" constant=\"true\"/>\n\t</listOfReactants>\n\t<listOfProducts>\n\t\t<speciesReference species=\"ADP\" stoichiometry=\"1\" constant=\"true\"/>\n\t\t<speciesReference species=\"PI\" stoichiometry=\"1\" constant=\"true\"/>\n\t</listOfProducts>\n</reaction>\n")

553

# Biomass reactions

In [100]:
f.write("</listOfReactions>\n\n")

20

# List of Gene Products

In [101]:
f.write("<fbc:listOfGeneProducts>\n")

25

In [102]:
if rdfba:
    for index, value in rp['id'].items():
        f.write("  <fbc:geneProduct fbc:id=\""+value+"\" fbc:label=\""+rp['Gene'][index]+"\" fbc:associatedSpecies=\""+value+"\"/>\n")

f.write("</fbc:listOfGeneProducts>\n\n")

27

# List of Qualitative Species

In [103]:
if rdfba:
    f.write("<qual:listOfQualitativeSpecies>\n")
    for index, value in qualspec['id'].items():
        f.write("  <qual:qualitativeSpecies qual:id=\""+value+"\" qual:maxLevel=\""+str(qualspec['max'][index])+"\" qual:compartment=\"cytosol\" qual:constant=\"false\"/>\n")
    f.write("</qual:listOfQualitativeSpecies>\n\n")

# List of Rules

In [104]:
if rdfba:
    f.write("<listOfRules>\n")

# FBC Rules

In [105]:
# Lactate (LACUP) -- spontaneous, but regulated
if rdfba:
    qual_parameters.append('GL_neg')
    f.write("  <assignmentRule variable=\"T_LACUP_FBClower\">\n    <math xmlns=\"http://www.w3.org/1998/Math/MathML\">\n      <apply>\n        <times/>\n        <ci>epsilon_trans</ci>\n        <ci>GL_neg</ci>\n      </apply>\n    </math>\n  </assignmentRule>\n")         
    f.write("  <assignmentRule variable=\"T_LACUP_FBCupper\">\n    <math xmlns=\"http://www.w3.org/1998/Math/MathML\">\n      <apply>\n        <times/>\n        <ci>NaN</ci>\n        <ci>GL_neg</ci>\n      </apply>\n    </math>\n  </assignmentRule>\n")

# Qual Rules

In [106]:
if rdfba:
    for qual in qualspec['id']:
        f.write("<assignmentRule variable=\"" + qual + "\">\n  <math xmlns=\"http://www.w3.org/1998/Math/MathML\">\n    <ci>qual_con_" + qual + "</ci>\n  </math>\n</assignmentRule>\n")

# Boolean Rules

In [107]:
if rdfba:
    for index, value in rules['variable'].items():
        qual_parameters.append(value)
        f.write("<assignmentRule variable=\"" + value + "\">\n  <math xmlns=\"http://www.w3.org/1998/Math/MathML\">\n    <apply>\n      " + rules['relation'][index] + "\n")
        for i in rules['indicators'][index]:
            qual_parameters.append(i)
            f.write("      <ci>" + i + "</ci>\n")
        f.write("    </apply>\n  </math>\n</assignmentRule>\n")

In [108]:
if rdfba:
    f.write("</listOfRules>\n\n")

# Events

In [109]:
if rdfba:
    f.write("<listOfEvents>\n")
    for index, value in events['id'].items():
        f.write("<event id=\"" + value + "\" useValuesFromTriggerTime=\"true\">\n  <trigger persistent=\"true\" initialValue=\"true\">\n    <math xmlns=\"http://www.w3.org/1998/Math/MathML\">\n")
        f.write("      <apply>\n        <" + events['relation'][index] + "/>\n")
        if len(events['variable'][index]) > 1: # multiple variables
            f.write("        <apply>\n          <plus/>\n")
            for var in events['variable'][index]:
                f.write("          <ci>" + var + "</ci>\n")
            f.write("        </apply>\n")
        else:
            f.write("        <ci>" + events['variable'][index][0] + "</ci>\n")
        f.write("        <ci>"+events['threshold'][index]+"</ci>\n      </apply>\n    </math>\n  </trigger>\n  <listOfEventAssignments>\n")
        for i, assignment in enumerate(events['assignment'][index]):
            qual_parameters.append(assignment)
            f.write("    <eventAssignment variable=\""+assignment+"\">\n      <math xmlns=\"http://www.w3.org/1998/Math/MathML\">\n        <cn type=\"integer\">"+events['value'][index][i]+"</cn>\n      </math>\n    </eventAssignment>\n")
        f.write("  </listOfEventAssignments>\n</event>\n")
    f.write("</listOfEvents>\n\n")

# List of Parameters

In [110]:
# filter duplicates
qual_parameters = list(set(qual_parameters))

if rdfba:
    f.write("<listOfParameters>\n")
    for index, value in parameters.items():
        f.write("  <parameter id=\"" + index + "\" constant=\"true\" value=\"" + str(value) + "\"/>\n")
    for fbc_param in fbc_parameters:
        f.write("  <parameter id=\"" + fbc_param + "\" value=\"NaN\" constant=\"false\" />\n")
    for qual_param in qual_parameters:
        f.write("  <parameter id=\"" + qual_param + "\" value=\"NaN\" constant=\"false\" />\n")
    f.write("</listOfParameters>\n")

# Last Lines

In [111]:
f.write("</model>\n")
f.write("</sbml>")
f.close()